# 노트북에서 모델 학습 및 서빙 API 생성 파이프라인 만들기

## 1. 라이브러리 추가 및 데이터 확인

In [ ]:
import requests
import os
import uuid
from kakaocloud_kbm import KbmPipelineClient
import kfp.compiler as compiler
from kfp import dsl
from kfp.dsl import ContainerOp, pipeline
from kfp import components
from kfp.components import create_component_from_func
import gzip
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO

# Fashion MNIST 데이터셋 URL
t10k_images_url = 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-images-idx3-ubyte.gz'
t10k_labels_url = 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-labels-idx1-ubyte.gz'

def download_and_extract(url):
    response = requests.get(url)
    with gzip.open(BytesIO(response.content), 'rb') as f:
        return np.frombuffer(f.read(), np.uint8)

t10k_images = download_and_extract(t10k_images_url)[16:].reshape(-1, 28, 28)
t10k_labels = download_and_extract(t10k_labels_url)[8:]

label_names = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", 
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]

def show_images(images, labels, label_names, rows=5, cols=5):
    fig, axes = plt.subplots(rows, cols, figsize=(10, 10))
    axes = axes.flatten()
    
    for img, ax, lbl in zip(images, axes, labels):
        ax.imshow(img, cmap='gray')
        ax.set_title(label_names[lbl])
        ax.axis('off')

    plt.tight_layout()
    plt.show()

show_images(t10k_images[:25], t10k_labels[:25], label_names)


## 2. 환경 변수 설정

In [ ]:
os.environ["KUBEFLOW_HOST"] = "{https://도메인주소}" #도메인 주소 마지막에 '/'를 넣지 마세요. ex) https://kakaocloud-edu.com
os.environ["KUBEFLOW_USERNAME"] = "{계정 아이디}"
os.environ["KUBEFLOW_PASSWORD"] = "{계정 비밀번호}"

# 환경변수들 준비
KBM_NAMESPACE = os.environ['NB_PREFIX'].split('/')[2]
TRAIN_PATH = 'fmnist_serve_model'
TRAIN_CR_IMAGE = "bigdata-150.kr-central-2.kcr.dev/kc-kubeflow/kmlp-pytorch:1.0.0.py36.cuda"
TASK_UUID = uuid.uuid1().hex[:8]
PVC_NAME = f"test-fmnist-pvc-{TASK_UUID}"
MODEL_NAME = f"torch-model-{TASK_UUID}"
KBM_MODEL_SERV_NAME = f"torchserve-{TASK_UUID}"
EPOCH_NUM = 3

print(f"Namespace : {KBM_NAMESPACE}")
print(f"Train Path : {TRAIN_PATH}")
print(f"Image for Training : {TRAIN_CR_IMAGE}")
print(f"Model Name : {MODEL_NAME}")
print(f"Model PVC Name : {PVC_NAME}")
print(f"Model Server Name : {KBM_MODEL_SERV_NAME}")
print(f"Number of Epochs : {EPOCH_NUM}")

# 학습을 위한 폴더 생성
os.makedirs(TRAIN_PATH, exist_ok=True)

## 3. 파이프라인 컴포넌트 빌드하기

### 3-1. 데이터셋 준비 컴포넌트: Fashion MNIST 데이터셋을 다운로드하는 함수 정의

In [ ]:
def download_fashion_mnist(
    t10k_images_url: str = 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-images-idx3-ubyte.gz',
    t10k_labels_url: str = 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-labels-idx1-ubyte.gz',
    train_images_url: str = 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-images-idx3-ubyte.gz',
    train_labels_url: str = 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-labels-idx1-ubyte.gz'
):
    import os
    import requests
    import gzip

    def download_and_save(url, output_path):
        response = requests.get(url, stream=True)
        with open(output_path, 'wb') as f:
            f.write(response.content)
        with gzip.open(output_path, 'rb') as f_in:
            with open(output_path.rstrip('.gz'), 'wb') as f_out:
                f_out.write(f_in.read())

    os.makedirs('/pvc', exist_ok=True)
    download_and_save(t10k_images_url, '/pvc/t10k-images-idx3-ubyte.gz')
    download_and_save(t10k_labels_url, '/pvc/t10k-labels-idx1-ubyte.gz')
    download_and_save(train_images_url, '/pvc/train-images-idx3-ubyte.gz')
    download_and_save(train_labels_url, '/pvc/train-labels-idx1-ubyte.gz')

    return '/pvc'

dataset_op = create_component_from_func(
    download_fashion_mnist,
    output_component_file=f'{TRAIN_PATH}/data_component.yaml',
    base_image='python:3.8',
    packages_to_install=['requests']
)

In [ ]:
%%writefile {TRAIN_PATH}/dataset_component.yaml
name: Fashion MNIST Dataset
description: |
  Fashion MNIST Dataset: https://github.com/zalandoresearch/fashion-mnist
metadata:
  annotations:
    author: KiC Bigdata <bigdata.platform@kakaoenterprise.com>
inputs:
- {name: t10k_images_url, type: String, default: 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-images-idx3-ubyte.gz'}
- {name: t10k_labels_url, type: String, default: 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-labels-idx1-ubyte.gz'}
- {name: train_images_url, type: String, default: 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-images-idx3-ubyte.gz'}
- {name: train_labels_url, type: String, default: 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-labels-idx1-ubyte.gz'}
implementation:
  container:
    image: curlimages/curl
    command:
    - sh
    - -c
    - |
      set -e -x -o pipefail
      curl -L "$0" --output "/pvc/t10k-images-idx3-ubyte.gz"
      curl -L "$1" --output "/pvc/t10k-labels-idx1-ubyte.gz"
      curl -L "$2" --output "/pvc/train-images-idx3-ubyte.gz"
      curl -L "$3" --output "/pvc/train-labels-idx1-ubyte.gz"
      if [ -s /pvc/t10k-images-idx3-ubyte.gz ] && [ -s /pvc/t10k-labels-idx1-ubyte.gz ] && [ -s /pvc/train-images-idx3-ubyte.gz ] && [ -s /pvc/train-labels-idx1-ubyte.gz ]; then
        gzip -d /pvc/t10k-images-idx3-ubyte.gz
        gzip -d /pvc/t10k-labels-idx1-ubyte.gz
        gzip -d /pvc/train-images-idx3-ubyte.gz
        gzip -d /pvc/train-labels-idx1-ubyte.gz
      else
        echo "Download failed, exiting."
        exit 1
      fi
    - {inputValue: t10k_images_url}
    - {inputValue: t10k_labels_url}
    - {inputValue: train_images_url}
    - {inputValue: train_labels_url}


### 3-2. Fashion MNIST 모델 학습 및 서빙 구성

In [ ]:
def train_fmnist(
    epoch_num: str,
    model_name: str,
    train_images_path: str,
    train_labels_path: str,
    test_images_path: str,
    test_labels_path: str
):
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torchvision import transforms
    from torch.utils.data import DataLoader, Dataset
    from PIL import Image
    import os
    import numpy as np

    class FashionMNISTDataset(Dataset):
        def __init__(self, images_path, labels_path, transform=None):
            self.images = self._read_images(images_path)
            self.labels = self._read_labels(labels_path)
            self.transform = transform

        def _read_images(self, path):
            with open(path, 'rb') as f:
                images = np.frombuffer(f.read(), np.uint8, offset=16)
            images = images.reshape(-1, 28, 28, 1)  # Ensure the shape is [num_samples, 28, 28, 1]
            return images

        def _read_labels(self, path):
            with open(path, 'rb') as f:
                labels = np.frombuffer(f.read(), np.uint8, offset=8)
            return labels

        def __len__(self):
            return len(self.labels)

        def __getitem__(self, idx):
            image = self.images[idx]
            label = self.labels[idx]
            image = Image.fromarray(image.squeeze(), mode='L')  # Convert to PIL Image
            if self.transform:
                image = self.transform(image)
            return image, torch.tensor(label, dtype=torch.long)  # Convert label to LongTensor

    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
            self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
            self.fc1 = nn.Linear(64 * 7 * 7, 128)
            self.fc2 = nn.Linear(128, 10)

        def forward(self, x):
            x = self.pool(torch.relu(self.conv1(x)))
            x = self.pool(torch.relu(self.conv2(x)))
            x = x.view(-1, 64 * 7 * 7)
            x = torch.relu(self.fc1(x))
            x = self.fc2(x)
            return x

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    
    train_dataset = FashionMNISTDataset(train_images_path, train_labels_path, transform=transform)
    test_dataset = FashionMNISTDataset(test_images_path, test_labels_path, transform=transform)
    trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    testloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    # Model Training
    model = Net()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(int(epoch_num)):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:    
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0

    print('Finished Training')
    os.makedirs('/pvc/fmnist_model', exist_ok=True)
    torch.save(model.state_dict(), '/pvc/fmnist_model/fmnist_cnn.pth')

    # Save handler.py
    handler_code = """
from ts.torch_handler.base_handler import BaseHandler
import torch
import torchvision.transforms as transforms
from PIL import Image
import io
import logging
import base64

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter('mylog - %(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

class ImageClassifierHandler(BaseHandler):
    def __init__(self):
        super(ImageClassifierHandler, self).__init__()
        self.initialized = False

    def initialize(self, context):
        import torch.nn as nn  # add import here
        logger.info("Initializing handler...")

        class Net(nn.Module):
            def __init__(self):
                super(Net, self).__init__()
                self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
                self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
                self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
                self.fc1 = nn.Linear(64 * 7 * 7, 128)
                self.fc2 = nn.Linear(128, 10)

            def forward(self, x):
                x = self.pool(torch.relu(self.conv1(x)))
                x = self.pool(torch.relu(self.conv2(x)))
                x = x.view(-1, 64 * 7 * 7)
                x = torch.relu(self.fc1(x))
                x = self.fc2(x)
                return x

        self.manifest = context.manifest
        properties = context.system_properties
        model_dir = properties.get("model_dir")
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        model_path = model_dir + '/fmnist_cnn.pth'
        logger.info(f"Loading model from {model_path}")
        try:
            self.model = Net()
            self.model.load_state_dict(torch.load(model_path, map_location=self.device))
            self.model.to(self.device)
            self.model.eval()
            logger.info("Model loaded successfully.")
        except Exception as e:
            logger.error(f"Error loading model: {e}")
            raise e

        self.image_processing = transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize((28, 28)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
        self.initialized = True
        logger.info("Handler initialized successfully.")

    def preprocess(self, data):
        logger.info("Preprocessing input data...")
        try:
            image = data[0].get("data") or data[0].get("body")
            if isinstance(image, str):
                image = io.BytesIO(base64.b64decode(image))
            else:
                image = io.BytesIO(image)
            image = Image.open(image)
            image = self.image_processing(image)
            logger.info("Preprocessing completed.")
            return image.unsqueeze(0).to(self.device)
        except Exception as e:
            logger.error(f"Error in preprocessing: {e}")
            raise e

    def inference(self, img):
        logger.info("Running inference...")
        try:
            with torch.no_grad():
                output = self.model(img)
                _, predicted = torch.max(output.data, 1)
            logger.info(f"Inference completed. Prediction: {predicted.item()}")
            return predicted
        except Exception as e:
            logger.error(f"Error in inference: {e}")
            raise e

    def postprocess(self, inference_output):
        logger.info("Postprocessing inference output...")
        try:
            result = [int(inference_output[0])]
            logger.info(f"Postprocessing completed. Result: {result}")
            return result
        except Exception as e:
            logger.error(f"Error in postprocessing: {e}")
            raise e

    def handle(self, data, context):
        logger.info("Handling request...")
        try:
            if not self.initialized:
                self.initialize(context)
            data = self.preprocess(data)
            data = self.inference(data)
            data = self.postprocess(data)
            logger.info("Request handled successfully.")
            return data
        except Exception as e:
            logger.error(f"Error in handle: {e}")
            raise e
"""

    with open("/pvc/fmnist_model/handler.py", "w") as f:
        f.write(handler_code)
        
    
    # config for torchserve
    import json
    config = dict(
        inference_address="http://0.0.0.0:8085",
        management_address="http://0.0.0.0:8085",
        metrics_address="http://0.0.0.0:8082",
        grpc_inference_port=7070,
        grpc_management_port=7071,
        enable_envvars_config="true",
        install_py_dep_per_model="true",
        model_store="/mnt/pvc/fmnist_model/model-store",
        model_snapshot=json.dumps({
            "name": "startup.cfg",
            "modelCount": 1,
            "models": {
                f"{model_name}": {  # Model Name
                    "1.0": {
                        "defaultVersion": "true",
                        "marName": f"{model_name}.mar",
                        "minWorkers": 1,
                        "maxWorkers": 5,
                        "batchSize": 1,
                        "maxBatchDelay": 10,
                        "responseTimeout": 60,
                    }
                }
            },
        }),
    )
    # creating config & config folder
    if not os.path.exists("/pvc/fmnist_model/config"):
        os.mkdir("/pvc/fmnist_model/config")
        
    with open("/pvc/fmnist_model/config/config.properties", "w") as f:
        for i, j in config.items():
            f.write(f"{i}={j}\n")
            
train_fmnist_op = components.create_component_from_func(
    train_fmnist, 
    output_component_file=f'{TRAIN_PATH}/train_component.yaml',
    base_image=TRAIN_CR_IMAGE
)

### 3-3. 서빙을 위한 MAR 파일 생성 컴포넌트

In [ ]:
def create_marfile():
    return dsl.ContainerOp(
        name="Creating Marfile",
        command=["/bin/sh"],
        image="python:3.9",
        arguments=[
            "-c",
            f"cd /pvc/fmnist_model; pip install torchserve torch-model-archiver torch-workflow-archiver; torch-model-archiver --model-name {MODEL_NAME} --version 1.0 --serialized-file fmnist_cnn.pth --handler handler.py --force; mkdir model-store; mv -f {MODEL_NAME}.mar model-store"
        ],
    )

### 3-4. KServe 컴포넌트 YAML 파일 작성

In [ ]:
%%writefile {TRAIN_PATH}/kserve_component.yaml
name: Serve a model with KServe 
description: Serve Models using KServe 
inputs:
  - {name: Action,                    type: String, default: 'create',     description: 'Action to execute on KServe'}
  - {name: Model Name,                type: String, default: '',           description: 'Name to give to the deployed model'}
  - {name: Model URI,                 type: String, default: '',           description: 'Path of the S3 or GCS compatible directory containing the model.'}
  - {name: Canary Traffic Percent,    type: String, default: '100',        description: 'The traffic split percentage between the candidate model and the last ready model'}
  - {name: Namespace,                 type: String, default: '',           description: 'Kubernetes namespace where the KServe service is deployed.'}
  - {name: Framework,                 type: String, default: '',           description: 'Machine Learning Framework for Model Serving.'}
  - {name: Custom Model Spec,         type: String, default: '{}',         description: 'Custom model runtime container spec in JSON'}
  - {name: Autoscaling Target,        type: String, default: '0',          description: 'Autoscaling Target Number'}
  - {name: Service Account,           type: String, default: '',           description: 'ServiceAccount to use to run the InferenceService pod'}
  - {name: Enable Istio Sidecar,      type: Bool,   default: 'True',       description: 'Whether to enable istio sidecar injection'}
  - {name: InferenceService YAML,     type: String, default: '{}',         description: 'Raw InferenceService serialized YAML for deployment'}
  - {name: Watch Timeout,             type: String, default: '300',        description: "Timeout seconds for watching until InferenceService becomes ready."}
  - {name: Min Replicas,              type: String, default: '-1',         description: 'Minimum number of InferenceService replicas'}
  - {name: Max Replicas,              type: String, default: '-1',         description: 'Maximum number of InferenceService replicas'}
  - {name: Request Timeout,           type: String, default: '60',         description: "Specifies the number of seconds to wait before timing out a request to the component."}
  - {name: Enable ISVC Status,        type: Bool,   default: 'True',       description: "Specifies whether to store the inference service status as the output parameter"}

outputs:
  - {name: InferenceService Status,   type: String,                        description: 'Status JSON output of InferenceService'}
implementation:
  container:
    image: bigdata.kr-central-1.kcr.dev/mlops-pipelines/kserve-component:v0.7.0.kbm.1c
    command: ['python']
    args: [
      -u, kservedeployer.py,
      --action,                 {inputValue: Action},
      --model-name,             {inputValue: Model Name},
      --model-uri,              {inputValue: Model URI},
      --canary-traffic-percent, {inputValue: Canary Traffic Percent},
      --namespace,              {inputValue: Namespace},
      --framework,              {inputValue: Framework},
      --custom-model-spec,      {inputValue: Custom Model Spec},
      --autoscaling-target,     {inputValue: Autoscaling Target},
      --service-account,        {inputValue: Service Account},
      --enable-istio-sidecar,   {inputValue: Enable Istio Sidecar},
      --output-path,            {outputPath: InferenceService Status},
      --inferenceservice-yaml,  {inputValue: InferenceService YAML},
      --watch-timeout,          {inputValue: Watch Timeout},
      --min-replicas,           {inputValue: Min Replicas},
      --max-replicas,           {inputValue: Max Replicas},
      --request-timeout,        {inputValue: Request Timeout},
      --enable-isvc-status,     {inputValue: Enable ISVC Status}
    ]


### 3-5. KServe 인퍼런스 모델 생성 컴포넌트

In [ ]:
from kfp.components import load_component_from_file

def create_inference_model():
    kserve_op = load_component_from_file(f'{TRAIN_PATH}/kserve_component.yaml')
    
    model_name = KBM_MODEL_SERV_NAME
    namespace = KBM_NAMESPACE
    model_uri = f"pvc://{PVC_NAME}/fmnist_model"
    framework="pytorch"
    
    opt = kserve_op(action="apply",
              model_name=model_name,
              model_uri=model_uri,
              namespace=namespace,
              framework=framework)
    
    opt.set_cpu_limit(cpu="2").set_memory_limit(memory="4G")
    return opt

## 4. 파이프라인 생성

In [ ]:
@dsl.pipeline(
    name="Fashion MNIST Model Pipeline"
)
def fmnist_model_pipeline(
    t10k_images_url: str = 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-images-idx3-ubyte.gz',
    t10k_labels_url: str = 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-labels-idx1-ubyte.gz',
    train_images_url: str = 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-images-idx3-ubyte.gz',
    train_labels_url: str = 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-labels-idx1-ubyte.gz',
    model_name: str = "torch-model",
    epoch_num: str = "4"
):
    
    vop = dsl.VolumeOp(
        name="volume_creation",
        resource_name=PVC_NAME,
        generate_unique_name=False,
        size="10Gi",
        modes=dsl.VOLUME_MODE_RWO
    )
    
    download_data = dataset_op(
        t10k_images_url=t10k_images_url,
        t10k_labels_url=t10k_labels_url,
        train_images_url=train_images_url,
        train_labels_url=train_labels_url
    ).add_pvolumes({"/pvc": vop.volume})
    download_data.set_cpu_limit(cpu="1").set_memory_limit(memory="2G")

    model_train = train_fmnist_op(
        epoch_num,
        model_name,
        '/pvc/train-images-idx3-ubyte',
        '/pvc/train-labels-idx1-ubyte',
        '/pvc/t10k-images-idx3-ubyte',
        '/pvc/t10k-labels-idx1-ubyte'
    ).add_node_selector_constraint(
        'nvidia.com/gpu.present', 'true' # GPU 사용 활성화
    ).add_pvolumes({"/pvc": vop.volume})
    
    model_train.add_resource_limit(
        "nvidia.com/mig-1g.10gb", "1"
    )
    model_train.set_cpu_limit(cpu="4").set_memory_limit(memory="8G")
    model_train.set_display_name("Training Fashion MNIST Model")
    model_train.after(download_data)
    
    marfile = create_marfile()
    marfile.add_pvolumes({"/pvc": vop.volume})
    marfile.set_display_name("Creating Marfile")
    marfile.execution_options.caching_strategy.max_cache_staleness = "P0D" # cache 사용않는 옵션
    marfile.set_cpu_limit(cpu="1").set_memory_limit(memory="2G")
    marfile.after(model_train)

    inference_model = create_inference_model()
    inference_model.add_pvolumes({"/pvc": vop.volume})
    inference_model.set_cpu_limit(cpu="4").set_memory_limit(memory="8G")
    inference_model.after(marfile)

## 5. 파이프라인 실행

In [ ]:
experiment_name = fmnist_model_pipeline.__name__ + ' test experiment'
run_name = fmnist_model_pipeline.__name__ + ' run'

print("experiment_name: " + experiment_name)
print("run_name: " + run_name)

arguments = {
    "model_name": MODEL_NAME,
    "epoch_num": str(EPOCH_NUM)
}

client = KbmPipelineClient()
client.create_run_from_pipeline_func(
    fmnist_model_pipeline, 
    experiment_name=experiment_name, 
    run_name=run_name, 
    arguments=arguments
)

## 6. 모델 서빙 API 테스트

### 6-1. 모델 서빙 API 테스트 이미지 준비

In [ ]:
import requests
import gzip
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO

# Fashion MNIST 데이터셋 URL
t10k_images_url = 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-images-idx3-ubyte.gz'
t10k_labels_url = 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-labels-idx1-ubyte.gz'

# 데이터 다운로드 및 압축 해제 함수
def download_and_extract(url):
    response = requests.get(url)
    with gzip.open(BytesIO(response.content), 'rb') as f:
        return np.frombuffer(f.read(), np.uint8)

# 데이터 다운로드 및 로드
t10k_images = download_and_extract(t10k_images_url)[16:].reshape(-1, 28, 28)
t10k_labels = download_and_extract(t10k_labels_url)[8:]

# 라벨 이름 정의
label_names = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", 
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]

# 랜덤 이미지 선택 및 출력 함수
def get_random_image(images, labels):
    random_idx = np.random.randint(len(images))
    random_image = images[random_idx]
    random_label = labels[random_idx]
    return random_image, random_label

def show_image(image, label, label_names):
    plt.imshow(image, cmap='gray')
    plt.title(f'Label: {label} ({label_names[label]})')
    plt.axis('off')
    plt.show()

# 랜덤 이미지 선택
random_image, random_label = get_random_image(t10k_images, t10k_labels)

# 랜덤 이미지 출력
show_image(random_image, random_label, label_names)

### 6-2. 모델 서빙 API 테스트

In [ ]:
import base64
import io
import requests
from PIL import Image
import os

# 설정 변수
KBM_NAMESPACE = os.environ['NB_PREFIX'].split('/')[2]
KUBEFLOW_HOST = os.environ["KUBEFLOW_HOST"]
KUBEFLOW_USERNAME = os.environ["KUBEFLOW_USERNAME"]
KUBEFLOW_PASSWORD = os.environ["KUBEFLOW_PASSWORD"]
MODEL_NAME = f"torch-model-{TASK_UUID}"
KBM_MODEL_SERV_NAME = f"torchserve-{TASK_UUID}"

def get_authenticated_session():
    session = requests.Session()
    if KUBEFLOW_HOST.startswith("https"):
        _kargs = {"verify": False}
    else:
        _kargs = {}

    response = session.get(KUBEFLOW_HOST, **_kargs)
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    session.post(response.url, headers=headers, data={"login": KUBEFLOW_USERNAME, "password": KUBEFLOW_PASSWORD})
    session_cookie = session.cookies.get_dict()["authservice_session"]
    return session, session_cookie, _kargs

def encode_image_to_base64(image):
    buffered = io.BytesIO()
    image = Image.fromarray(image)
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

def send_test_request(image, session, session_cookie, _kargs):
    input_image_data = encode_image_to_base64(image)
    data = {"instances": [{"data": input_image_data}]}
    endpoint = f"{KUBEFLOW_HOST}/v1/models/{MODEL_NAME}:predict"
    print(f"endpoint: {endpoint}")

    response = session.post(
        url=endpoint,
        cookies={'authservice_session': session_cookie},
        headers={"Host": f"{KBM_MODEL_SERV_NAME}.{KBM_NAMESPACE}.{KUBEFLOW_HOST.split('/')[2]}"},
        json=data,
        **_kargs
    )
    
    try:
        response_json = response.json()
    except ValueError:
        print(f"Error: Unable to parse JSON response. Response text: {response.text}")
        return None
    
    return response_json

session, session_cookie, _kargs = get_authenticated_session()

# 사용자 이미지 로드
#image = np.array(Image.open('dress.png'))
#random_label = None

response = send_test_request(random_image, session, session_cookie, _kargs)
if response:
    predicted_label = response['predictions'][0]
    print(predicted_label)
    if random_label is not None:
        print(f"입력 라벨: {label_names[random_label]}")
    print(f"예측 라벨: {label_names[predicted_label]}")
else:
    print("Failed to get prediction from the model server.")


In [ ]:
!kubectl get inferenceservices -n kbm-u-kubeflow-tutorial -o yaml

In [ ]:
!kubectl get po -n kbm-u-kubeflow-tutorial

In [ ]:
!kubectl get po torchserve-401a30c0-predictor-default-00001-deployment-6674nngx -n kbm-u-kubeflow-tutorial -o yaml